In [122]:
import numpy as np
import pyvista as pv
import torch
from numba import jit, prange
from scipy.spatial import cKDTree

In [2]:
etching = np.load('./bosch_data_1012_ratio08_trench_condition5_300wide/bosch_sf_step_12_sf.npy')

In [123]:
etching = np.load('./test_depoFloat.npy')

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:50, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [4]:
def removeFloat(film):  # fast scanZ
    filmC = film[:,:,:,0]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,1] != 0

    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = np.zeros_like(filmC, dtype=bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & ~neighbors

    # 将孤立的单元格设为0
    film[condition, :] = 0

    return film

In [ ]:
    # def scanZ(self, film): # fast scanZ
    #     film = torch.Tensor(film)
    #     xshape, yshape, zshape = film.shape
    #     self.zshape = zshape
    #     # 初始化一个全零的表面稀疏张量
    #     surface_sparse = torch.zeros((xshape, yshape, zshape))
        
    #     # 获取当前平面与前后平面的布尔索引
    #     current_plane = film != 0

    #     # 获取周围邻居的布尔索引
    #     neighbors = torch.zeros_like(film, dtype=torch.bool)
        
    #     neighbors[1:, :, :] |= film[:-1, :, :] == 0  # 上面
    #     neighbors[:-1, :, :] |= film[1:, :, :] == 0  # 下面
    #     neighbors[:, 1:, :] |= film[:, :-1, :] == 0  # 左边
    #     neighbors[:, :-1, :] |= film[:, 1:, :] == 0  # 右边
    #     neighbors[:, :, 1:] |= film[:, :, :-1] == 0  # 前面
    #     neighbors[:, :, :-1] |= film[:, :, 1:] == 0  # 后面
        
    #     # 获取满足条件的索引
    #     condition = current_plane & neighbors
        
    #     # 更新表面稀疏张量
    #     surface_sparse[condition] = 1
        
    #     return surface_sparse.to_sparse()

In [29]:
def getFloat_pointcloud(film):  # fast scanZ
    sumFilm = np.sum(film, axis=-1)
    sumFilm = torch.Tensor(sumFilm)
    film = torch.Tensor(film)
    filmC = film[:,:,:,0]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,1] != 0
    # 初始化一个全零的表面稀疏张量
    surface_sparse = torch.zeros_like(sumFilm)
    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = torch.zeros_like(filmC, dtype=torch.bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & ~neighbors

    surface_sparse[condition] = 1

    points = surface_sparse.to_sparse().indices().T
    return points

In [ ]:
def removeFloat(film):  # fast scanZ
    filmC = film[:,:,:,0]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,1] != 0

    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = np.zeros_like(filmC, dtype=bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & ~neighbors

    # 将孤立的单元格设为0
    film[condition, :] = 0

    return film

In [ ]:
def getFloat_pointcloud(film):  # fast scanZ
    sumFilm = np.sum(film, axis=-1)
    sumFilm = torch.Tensor(sumFilm)
    film = torch.Tensor(film)
    filmC = film[:,:,:,0]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,1] != 0
    # 初始化一个全零的表面稀疏张量
    surface_sparse = torch.zeros_like(sumFilm)
    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = torch.zeros_like(filmC, dtype=torch.bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & ~neighbors

    surface_sparse[condition] = 1

    points = surface_sparse.to_sparse().indices().T
    return points

In [ ]:
geom = pv.Box()
etching = np.load('./test_depoFloat.npy')
# substrute = torch.Tensor(film[:, :, :]==10).to_sparse()
# substrute = substrute.indices().numpy().T

cyan = torch.Tensor(np.array(etching[:, :, :,0]!=0)).to_sparse()
cyan = cyan.indices().numpy().T

# submesh = pv.PolyData(substrute)
# submesh["radius"] = np.ones(substrute.shape[0])*0.5

cyanmesh = pv.PolyData(cyan)
cyanmesh["radius"] = np.ones(cyan.shape[0])*0.5
# Progress bar is a new feature on master branch
cyanglyphed = cyanmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
# subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
# p.add_mesh(subglyphed, color='gray')
p.add_mesh(cyanglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()

In [168]:
def scanDepoFloat(film): # fast scanZ
    film = torch.Tensor(film)
    sumFilm = torch.sum(film, axis=-1)
    # sumFilm = torch.Tensor(sumFilm)

    filmC = film[:,:,:,0]
    # 初始化一个全零的表面稀疏张量
    surface_sparse = torch.zeros_like(sumFilm)
    surface_Float = torch.zeros_like(sumFilm)

    # 获取当前平面与前后平面的布尔索引
    current_plane = sumFilm == 0
    current_Float = torch.logical_and(film[:,:,:,0] > 0, film[:,:,:,0] < 1)
    # print(current_Float)
    # 获取周围邻居的布尔索引
    neighbors_plane = torch.zeros_like(filmC, dtype=torch.bool)
    
    neighbors_plane[1:, :, :] |= filmC[:-1, :, :] >= 9  # 上面
    neighbors_plane[:-1, :, :] |= filmC[1:, :, :] >= 9  # 下面
    neighbors_plane[:, 1:, :] |= filmC[:, :-1, :] >= 9  # 左边
    neighbors_plane[:, :-1, :] |= filmC[:, 1:, :] >= 9  # 右边
    neighbors_plane[:, :, 1:] |= filmC[:, :, :-1] >= 9  # 前面
    neighbors_plane[:, :, :-1] |= filmC[:, :, 1:] >= 9  # 后面

    # 获取周围邻居的布尔索引
    neighbors_float = torch.zeros_like(filmC, dtype=torch.bool)
    
    neighbors_float[1:, :, :] |= filmC[:-1, :, :] == 0  # 上面
    neighbors_float[:-1, :, :] |= filmC[1:, :, :] == 0  # 下面
    neighbors_float[:, 1:, :] |= filmC[:, :-1, :] == 0  # 左边
    neighbors_float[:, :-1, :] |= filmC[:, 1:, :] == 0  # 右边
    neighbors_float[:, :, 1:] |= filmC[:, :, :-1] == 0  # 前面
    neighbors_float[:, :, :-1] |= filmC[:, :, 1:] == 0  # 后面
    
    # 获取满足条件的索引
    condition = current_plane & neighbors_plane
    condition_float = current_Float & neighbors_float
    # 更新表面稀疏张量
    surface_sparse[condition] = 1
    surface_Float[condition_float] = 1

    points = surface_Float.to_sparse().indices().T

    return surface_sparse.numpy(), points

In [ ]:
float_point = scanDepoFloat(etching)[1]
float_point.shape

In [ ]:
float_point = scanDepoFloat(etching)[1]

point_cloud = pv.PolyData(float_point.numpy())
vectors = np.ones((float_point.shape[0], 3))

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=10000,
    factor=3,
)

# Display the arrowscyan


plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [185]:
def depoFloat(film):
    plane_point, Float_point = scanDepoFloat(film)
    # Float_point = getFloat_pointcloud(film)
    print(Float_point.shape)
    plane_tree = cKDTree(np.argwhere(plane_point == 1))
    # Float_point = getFloat_pointcloud(film)

    dd, ii = plane_tree.query(Float_point, k=1, workers=1)
    surface_indice = np.argwhere(plane_point)
    i1 = surface_indice[ii][:,0] #[particle, order, xyz]
    j1 = surface_indice[ii][:,1]
    k1 = surface_indice[ii][:,2]

    Float_point = Float_point.numpy()
    film[i1, j1, k1, 0] += film[Float_point[:, 0],Float_point[:, 1],Float_point[:, 2],0]
    # film[i1, j1, k1, 0]  = 10
    film[Float_point[:, 0],Float_point[:, 1],Float_point[:, 2],0] = 0

    return film

In [ ]:
etching[40, 100, 80:,0]

In [ ]:
etchingB[40, 100, 80:,0]

In [ ]:
etching = np.load('./test_depoFloat.npy')
etchingB = depoFloat(etching)

In [ ]:
geom = pv.Box()

substrute = torch.Tensor(np.logical_and(etchingB[:, :, :,0]>0, etchingB[:, :, :,0]<2)).to_sparse()
substrute = substrute.indices().numpy().T

cyan = torch.Tensor(np.array(etchingB[:, :, :,0]>=9)).to_sparse()
cyan = cyan.indices().numpy().T

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

cyanmesh = pv.PolyData(cyan)
cyanmesh["radius"] = np.ones(cyan.shape[0])*0.5
# Progress bar is a new feature on master branch
cyanglyphed = cyanmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
p.add_mesh(subglyphed, color='gray')
p.add_mesh(cyanglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
def depoFloat(film):
    plane_point = scanZ(film)
    Float_point = getFloat_pointcloud(film)
    
    plane_tree = cKDTree(np.argwhere(plane_point == 1))
    # Float_point = getFloat_pointcloud(film)

    dd, ii = plane_tree.query(Float_point, k=1, workers=1)
    surface_indice = np.argwhere(plane_point)
    i1 = surface_indice[ii][:,0] #[particle, order, xyz]
    j1 = surface_indice[ii][:,1]
    k1 = surface_indice[ii][:,2]

    Float_point = Float_point.numpy()
    film[i1, j1, k1, 1] += 20
    film[Float_point[:, 0],Float_point[:, 1],Float_point[:, 2],1] = 30

    return film

In [ ]:
etching = np.load('./bosch_data_1012_ratio08_trench_condition5_300wide/bosch_sf_step_12_sf.npy')
plane_point = scanZ(etching)
# print(plane_point.shape)


plane_tree = cKDTree(np.argwhere(plane_point == 1))
Float_point = getFloat_pointcloud(etching)
print(Float_point.shape)
# indice_all = np.zeros_like(pos.shape[0], dtype=np.bool_)
dd, ii = plane_tree.query(Float_point, k=1, workers=1)
surface_indice = np.argwhere(plane_point)
i1 = surface_indice[ii][:,0] #[particle, order, xyz]
j1 = surface_indice[ii][:,1]
k1 = surface_indice[ii][:,2]
print(ii.shape)
print('ii', ii)
print(surface_indice.shape)
# print(np.max(ii))
print(Float_point.shape)
print(etching.shape)
Float_point = Float_point.numpy()
print('Float_point',Float_point.shape)
print(surface_indice[ii].shape)
print(etching[i1, j1, k1, 1].shape)
# etching[i1, j1, k1, 1] += Float_point[:, 1]
# print(ii)
# etching[]

In [110]:
def depoFloat(film):
    plane_point = scanZ(film)

    plane_tree = cKDTree(np.argwhere(plane_point == 1))
    Float_point = getFloat_pointcloud(film)

    dd, ii = plane_tree.query(Float_point, k=1, workers=1)
    surface_indice = np.argwhere(plane_point)
    i1 = surface_indice[ii][:,0] #[particle, order, xyz]
    j1 = surface_indice[ii][:,1]
    k1 = surface_indice[ii][:,2]

    Float_point = Float_point.numpy()
    film[i1, j1, k1, 1] += 20
    film[Float_point[:, 0],Float_point[:, 1],Float_point[:, 2],1] = 30

    return film

In [120]:
etching = np.load('./bosch_data_1012_ratio08_trench_condition5_300wide/bosch_sf_step_12_sf.npy')
etching = depoFloat(etching)

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]==10, etching[:, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:, :, :,1]==20).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :, :,1]==30).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [5]:
etching = removeFloat(etching)

In [80]:
etchingB = np.copy(etching)

In [98]:
etching[i1, j1, k1, 1] += 10

In [101]:
etching[Float_point[:, 0],Float_point[:, 1],Float_point[:, 2],1] = 0

In [ ]:
np.allclose(etching, etchingB)

In [ ]:
etching.shape

In [ ]:
etchingB.shape

In [ ]:
np.allclose(etching[:, 1], etchingB[:, 1])

In [ ]:
np.allclose(etching[:, 0], etchingB[:, 0])

In [ ]:
np.allclose(etching[:, 2], etchingB[:, 2])

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]!=10, etching[:, :, :,1]!=0)).to_sparse()
# cyan = torch.Tensor(etching[:, 1] != etchingB[:, 1]).to_sparse()
cyan = torch.Tensor(etching[:, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()